In [ ]:
from etils import epath
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchio as tio
from torchsummary import summary
import plotly.express as px

from codebase.preprocessor.images import multi_modal_processor
from codebase.dataloader.images import multi_modal_dataloader
import codebase.terminology as term
import codebase.codebase_settings as cbs

%load_ext autoreload
%autoreload 2

<h3> Preprossing data </h3>

In [ ]:
data_folder = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data'
hecktor_processor_train = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder, phase=term.Phase.TRAIN, modalities=[term.Modality.CT, term.Modality.PET],
    reference=term.Modality.CT, problem_type=term.ProblemType.SEGMENTATION, desired_voxel_size=(1.0, 1.0, 2.0))

<h> Original data </h>

In [ ]:
# create raw subject
subject = hecktor_processor_train.create_subject('MDA-103')
subject

In [ ]:
print(subject['CT'].shape)
print(subject['CT'].orientation)
subject['CT'].plot()

In [ ]:
print(subject['PT'].shape)
subject['PT'].plot()
print(subject['PT'].data.max())
print(subject['PT'].data.min())
print(subject['PT'].data.mean())
print(subject['PT'].data.std())

In [ ]:
pt_data = subject['PT'].numpy().flatten()
ax = plt.hist(pt_data[pt_data > 0.05], bins=100)

In [ ]:
print(subject['LABEL'].shape)
print(subject['LABEL'].data.max())
subject['LABEL'].plot()

<h> Resample to reference (PET) </h>

In [ ]:
resampled_subject = hecktor_processor_train.resample_to_reference(subject=subject)

In [ ]:
print(resampled_subject['CT'].shape)
print(resampled_subject['CT'].spacing)
print(resampled_subject['PT'].spacing)
print(resampled_subject['LABEL'].spacing)
resampled_subject['CT'].plot()

In [ ]:
pt_data = resampled_subject['PT'].numpy().flatten()
ax = plt.hist(pt_data[pt_data > 0.05], bins=100)

<h> Apply post_normalization:

CT: Clamp + Intensity rescale

PET: Normalize to brain </h> 

In [ ]:
post_normalization = hecktor_processor_train.create_post_normalization()
post_subject = post_normalization(resampled_subject)

In [ ]:
ct_data = post_subject['CT'].numpy()
print(f'max: {np.max(ct_data)}')
print(f'min: {np.min(ct_data)}')
post_subject['CT'].plot()

In [ ]:
pt_data = post_subject['PT'].numpy()
print(f'max: {np.max(pt_data)}')
print(f'min: {np.min(pt_data)}')
post_subject['PT'].plot()

In [ ]:
pt_data[:, :, 0]

<h> Create body mask </h>

In [ ]:
body_thresholds = {term.Modality.CT: (0.2, 1), term.Modality.PET: (0.4, 1)}
post_subject = hecktor_processor_train.create_body_mask(post_subject, body_thresholds)

In [ ]:
print(post_subject['BODY'].shape)
post_subject['BODY'].plot()

In [ ]:
px.imshow(
    np.swapaxes(resampled_subject['PT'].numpy(), 1, 3),
    # zmin=[0, 0, 0],
    # zmax=[2000, 2000, 2000],
    animation_frame=1,
    # binary_string=gray_scale,
    labels={'animation_frame': 'slice'},
    facet_col=0,
    color_continuous_scale='Gray',
    width=500, height=500
)

<h> Cropping the images </h>

In [ ]:
cropped_subject = hecktor_processor_train.find_bounding_box_and_crop(post_subject, desired_xy_size=(256, 256))

In [ ]:
print(cropped_subject['CT'].shape)
print(cropped_subject['CT'].spacing)
cropped_subject['PT'].plot()

In [ ]:
(cropped_subject['BODY'].tensor)[..., 150].sum()

In [ ]:
px.imshow(
    np.swapaxes(pt_data, 1, 3),
    # zmin=[0, 0, 0],
    # zmax=[2000, 2000, 2000],
    animation_frame=1,
    # binary_string=gray_scale,
    labels={'animation_frame': 'slice'},
    facet_col=0,
    color_continuous_scale='Gray',
    width=500, height=500
)

In [ ]:
plt.hist(pt_data.flatten(), bins=100)

<h> Process and save data </h>

In [ ]:
n = hecktor_processor_train.preprocess_and_save(xy_size=(256, 256), weight_modality=term.Modality.PET, weight_threshold=0.5)

In [ ]:
hecktor_processor_valid = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder, phase=term.Phase.VALID, modalities=[term.Modality.CT, term.Modality.PET],
    reference=term.Modality.CT, problem_type=term.ProblemType.SEGMENTATION, desired_voxel_size=(1.0, 1.0, 2.0))
n = hecktor_processor_valid.preprocess_and_save(xy_size=(256, 256), weight_modality=term.Modality.PET, weight_threshold=0.5)

In [ ]:
hecktor_processor_test = multi_modal_processor.MultiModalProcessor(
    data_folder=data_folder, phase=term.Phase.TEST, modalities=[term.Modality.CT, term.Modality.PET],
    reference=term.Modality.CT, problem_type=term.ProblemType.SEGMENTATION, desired_voxel_size=1)
n = hecktor_processor_test.preprocess_and_save(xy_size=(256, 256), weight_modality=term.Modality.PET, weight_threshold=0.5)

<h3> Subvolume generation </h3>

In [ ]:
hecktor_processor_train.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)

In [ ]:
image = np.load(str(data_folder / 'processed_256x256/subvolume_32/train/images/CHUP-052_52__input.npy'))
label = np.load(str(data_folder / 'processed_256x256/subvolume_32/train/labels/CHUP-052_52__label.npy'))

In [ ]:
print(image.shape, label.shape)

In [ ]:
# all_imgs = [label.data[0, :, :, :].numpy(), label.data[1, :, :, :].numpy(), label.data[2, :, :, :].numpy(),]
all_imgs = np.swapaxes(label, 1, 3)
px.imshow(
    all_imgs,
    # zmin=[0, 0, 0],
    # zmax=[2000, 2000, 2000],
    animation_frame=1,
    # binary_string=gray_scale,
    labels={'animation_frame': 'slice'},
    facet_col=0,
    color_continuous_scale='Gray',
    width=500, height=500
)

In [ ]:
hecktor_processor_valid.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)

In [ ]:
hecktor_processor_test.create_and_save_subvolumes(data_path=data_folder / 'processed_256x256', 
                                             valid_channel=[0], subvolume_intervel=8, subvolume_size=32)